## Sentiment Analysis using RNN

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import tensorflow as tf
from tensorflow.keras.layers import SimpleRNN, LSTM, Dense, Dropout, Embedding, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences

import warnings
warnings.filterwarnings('ignore')


In [ ]:
data = pd.read_csv('C:/Users/ailaty/Desktop/Python Scripts/Amazon Product Review Sentiment Analysis using RNN-LSTM/AmazonReview.csv')

# Printing shape of the dataset
print(data.shape)
# printing columns and rows information
print(data.info())


In [ ]:
data.head()

In [ ]:
# looking for NULL values
print("Null Values:\n", data.isna().sum())

# dropping null values
data = data.dropna()

# again checking for NULL values
print("Null Values after dropping:\n", data.isna().sum())

In [ ]:
# count of unique values in Sentiment column
data['Sentiment'].value_counts()

Text Cleaning: In this step, we will clean the ‘reviews.text’ column. We will remove the unwanted HTML tags, brackets, or special characters that may be present in the texts. We will use Regex to clean the text.

In [ ]:
# downloading stopwords from nltk library
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Review text Cleaning
def clean_reviews(text):
	
	# removing html brackets and other square brackets from the string using regex
	regex = re.compile('<.*?>') # r'<.*?>'
	text = re.sub(regex, '', text)

	# removing special characters like @, #, $, etc
	pattern = re.compile('[^a-zA-z0-9\s]')
	text = re.sub(pattern,'',text)

	# removing numbers
	pattern = re.compile('\d+')
	text = re.sub(pattern,'',text)

	# converting text to lower case
	text = text.lower()
	
	# Tokenization of words
	text = word_tokenize(text)
	
	# Stop words removal
	text = [word for word in text if not word in stop_words]
	
	return text

# using the clean_reviews function on the dataset
data['Review'] = data['Review'].apply(clean_reviews)

In [ ]:
tokenizer = Tokenizer()

# converting all the reviews to list to pass it as a parameter to fit_on_texts
reviews_to_list = data['Review'].tolist()
tokenizer.fit_on_texts(reviews_to_list)

# Generating text sequences
text_sequences = np.array(tokenizer.texts_to_sequences(reviews_to_list))

# one hot encoding
data = pd.get_dummies(data, columns = ['Sentiment'])

# setting maximum words we want in an example
max_words = 500

# Generatin our X (input) to the model
# using pad_sequences and y (output)
X = pad_sequences(text_sequences, maxlen = max_words)
y = data[['Sentiment_1', 'Sentiment_2', 'Sentiment_3', 'Sentiment_4',
	'Sentiment_5']]
print(X.shape, y.shape)

In [ ]:
# Train Test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
# Creating a RNN model
rnn = Sequential(name="Simple_RNN")
rnn.add(Embedding(len(tokenizer.word_index)+1,
						max_words,
						input_length=max_words))

rnn.add(SimpleRNN(128,activation='relu',return_sequences=True))

rnn.add(SimpleRNN(64,activation='relu',return_sequences=False))

rnn.add(Dense(5, activation='softmax'))

# printing model summary
print(rnn.summary())